# SMPL Tutorial


### created by Yan Zhang @ ETH Zurich, 08.03.2022

This script is associated with the course Virtual Humans'22 SMPL tutorial, and is provided as is. For third-party libs like SMPL models, please follow their official license to use.



In [ ]:
import numpy as np
import open3d as o3d
import torch


# test whether GPU works
try:
  aa = torch.cuda.FloatTensor(3,3).normal_()
except:
  # raise OSError('-- cuda is not supported')
  pass

# specify device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


print('-- pytorch version={}'.format(torch.__version__))
print('-- device={}'.format(device))



: 

## load the body model, SMPL-X and VPoser


**You need to go to the official website, download the models, and save them to some places. For example,



In [6]:
smplx_model_path='../models_smplx_v1_1/models/'
vposer_path='../vposer_v1_0'

In [3]:
import human_body_prior

In [4]:
dir(human_body_prior.tools.model_loader)

AttributeError: module 'human_body_prior' has no attribute 'tools'

In [7]:
import smplx  # SMPL-X; pip install smplx
from human_body_prior.tools.model_loader import load_model


def get_smplx_model(gender, device):
    body_model_path = smplx_model_path
    body_model = smplx.create(body_model_path, 
                          model_type='smplx',        ## smpl, smpl+h, or smplx?
                          gender=gender, ext='npz',  ## file format 
                          num_pca_comps=12,          ## MANO hand pose pca component
                          create_global_orient=True, 
                          create_body_pose=True,
                          create_betas=True,
                          create_left_hand_pose=True,
                          create_right_hand_pose=True,
                          create_expression=True, 
                          create_jaw_pose=True,
                          create_leye_pose=True,
                          create_reye_pose=True,
                          create_transl=True,
                          batch_size=1               ## how many bodies in a batch?
                          )
    body_model.eval()
    return body_model

smplx_male = get_smplx_model(gender='male', device=device)
smplx_female = get_smplx_model(gender='female', device=device)
smplx_neutral = get_smplx_model(gender='neutral', device=device)


# vposer, _ = load_model(vposer_path, vp_model='snapshot')
# vposer.eval()


In [18]:
vposer, _ = load_model(vposer_path)

AssertionError: No checck points found at ../vposer_v1_0\snapshots

## visualize the body model

In [8]:

def draw_bodies(gender, betas, thetas):

    if gender=='male':
        bm = smplx_male
    elif gender=='female':
        bm = smplx_female
    else:
        bm = smplx_neutral


    ## read verts and face from smplx model
    params = {'betas': betas,
              'body_pose': thetas
              }


    verts = bm(return_verts=True, **params).vertices.detach().cpu().numpy()[0]
    faces = bm.faces

    ## put verts and face into open3d, and compute surface normal
    coord = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5)
    body = o3d.geometry.TriangleMesh()
    body.vertices = o3d.utility.Vector3dVector(verts)
    body.triangles = o3d.utility.Vector3iVector(faces)
    body.vertex_normals = o3d.utility.Vector3dVector([])
    body.triangle_normals = o3d.utility.Vector3dVector([])
    body.compute_vertex_normals()
    o3d.visualization.draw_geometries([body, coord])
    
    return None



In [11]:
gender = 'neutral'
betas = torch.FloatTensor(1,10).zero_()
# thetas = vposer.decode(torch.FloatTensor(1,32).normal_(), output_type='aa')
thetas = torch.FloatTensor(1, 21, 3).zero_()

draw_bodies(gender, betas, thetas)

In [63]:
dir(smplx_neutral)

['EXPRESSION_SPACE_DIM',
 'J_regressor',
 'NECK_IDX',
 'NUM_BODY_JOINTS',
 'NUM_FACE_JOINTS',
 'NUM_HAND_JOINTS',
 'NUM_JOINTS',
 'SHAPE_SPACE_DIM',
 'T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_call_impl',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_get_backward_hooks',
 '_get_name',
 '_is_full_backward_hook',
 '_load_from_state_dict',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_named_members',
 '_non_persistent_buffers_set',
 '_num_betas',
 '_num_expression_coeffs',
 '_parameters',
 '_register_l

In [49]:
thetas2 = torch.FloatTensor(1, 21, 3).zero_()
# thetas2[0, 0] = torch.tensor([-3.0, 0.0, 0.0])  # left foot forward (or back)
# thetas2[0, 0] = torch.tensor([0.0, 0.0, 2.0])
thetas2[0, 19] = torch.tensor([-1.0, 1.0, 1.0])
thetas2[0, 18] = torch.tensor([-1.0, 1.0, 1.0])
thetas2[0, 17] = torch.tensor([-1.0, 1.0, 1.0])
thetas2[0, 16] = torch.tensor([-1.0, 1.0, 1.0])

In [50]:
draw_bodies(gender, betas, thetas2)

In [70]:
smplx_neutral()

SMPLXOutput(vertices=tensor([[[ 0.0600,  0.2760, -0.0089],
         [ 0.0641,  0.2750, -0.0079],
         [ 0.0647,  0.2765, -0.0078],
         ...,
         [-0.0386,  0.3091,  0.0519],
         [-0.0365,  0.3095,  0.0507],
         [-0.0342,  0.3099,  0.0499]]], grad_fn=<AsStridedBackward0>), joints=tensor([[[ 3.1233e-03, -3.5141e-01,  1.2037e-02],
         [ 6.1313e-02, -4.4417e-01, -1.3965e-02],
         [-6.0144e-02, -4.5532e-01, -9.2138e-03],
         [ 3.6056e-04, -2.4152e-01, -1.5581e-02],
         [ 1.1601e-01, -8.2292e-01, -2.3361e-02],
         [-1.0435e-01, -8.1770e-01, -2.6038e-02],
         [ 9.8083e-03, -1.0966e-01, -2.1521e-02],
         [ 7.2555e-02, -1.2260e+00, -5.5237e-02],
         [-8.8937e-02, -1.2284e+00, -4.6230e-02],
         [-1.5222e-03, -5.7428e-02,  6.9258e-03],
         [ 1.1981e-01, -1.2840e+00,  6.2980e-02],
         [-1.2775e-01, -1.2868e+00,  7.2819e-02],
         [-1.3687e-02,  1.0774e-01, -2.4690e-02],
         [ 4.4842e-02,  2.7515e-02, -2.9465e-04